In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
import pymysql
from urllib.parse import quote
from datetime import datetime, timedelta


In [2]:
sint_username = "New-anamika"
sint_password = "amb0!2s)f*xL3-+@1d$cM45"
sint_host = 'live-si-db.c83fh8gbkqpw.ap-south-1.rds.amazonaws.com'
sint_port = "3306"
sint_db_name = "sinterface"
sint_pwd = quote(sint_password)
sint = f"mysql+mysqlconnector://{sint_username}:{sint_pwd}@{sint_host}:{sint_port}/{sint_db_name}"

In [3]:
spot_username = "anamika"
spot_password = 'Ma8(2Ha1@39"gI4$5&d!+gU'
spot_host = 'spotlight-googlelookerreadreplica1.c83fh8gbkqpw.ap-south-1.rds.amazonaws.com'
spot_port = "3306"
spot_db_name = "verification_myspotlight"
spot_pwd = quote(spot_password)
spot = f"mysql+mysqlconnector://{spot_username}:{spot_pwd}@{spot_host}:{spot_port}/{spot_db_name}"

In [4]:
sinterface = create_engine(sint)
spotlight = create_engine(spot)

In [ ]:
raw = pd.read_csv('D:/Ops_Productivity/Month wise/raw.csv')

In [ ]:
verify = pd.read_csv('D:/Ops_Productivity/Month wise/verified.csv')

In [ ]:
gmb= pd.read_csv('D:/Ops_Productivity/Month wise/GMB.csv')

In [ ]:
raw['Timeline'] = pd.to_datetime(raw['Timeline'])
verify['Timeline'] = pd.to_datetime(verify['Timeline'])
gmb['Timeline'] = pd.to_datetime(gmb['Timeline'])

In [ ]:
data1 = pd.merge(raw, verify, how = 'left', on = ['Timeline'] )

In [ ]:
data2 = pd.merge(data1, gmb, how = 'left', on = ['Timeline'])

In [ ]:
data2.to_csv('month_level_data.csv')

In [ ]:
raw_query = '''
SELECT 
upper(orderCode) as order_code,
`name` as verification_brand_name,
count(r.id) as raw_location,
DATE_FORMAT((case when is_uploded = 0 then (date(r.created_at)) else (date(JSON_EXTRACT(row_data, '$[0]')))  end),  '%Y-%m-01')as Timeline
FROM verification_myspotlight.raw_locations r
inner join brands on brands.id = r.brand_id
where is_deleted = 0 
group by order_code, Timeline
having Timeline between '2023-01-01' and '2024-12-01' and order_code is not null and  order_code != '';
'''

In [ ]:
verified_query = '''
SELECT 
upper(orderCode) as order_code,
count(CASE WHEN (call_status = 'Verification done' or call_status = 5) THEN ( vr.raw_location_id) END) as verified_location,
count(CASE WHEN (call_status = 'Closed' or call_status = 15) THEN (vr.raw_location_id) END) as verified_closed_location,
DATE_FORMAT(verifiedDate,  '%Y-%m-01') as Timeline
FROM verification_myspotlight.verified_locations vr
inner join brands on brands.id = vr.brand_id
where is_deleted = 0 and verifiedDate between '2023-01-01' and '2024-12-01'
group by order_code, Timeline
having order_code is not null and  order_code != '';
'''


In [ ]:
SI_Query = '''
select 
parent_id as master_outlet_id,
upper(orders.order_code) as order_code,
COUNT(CASE WHEN delete_outlet IS NULL AND close_outlet IS NULL and parent_id is not null THEN outlets.id END) AS SI_live,
count(case when delete_outlet = 1 and close_outlet is  null then (outlets.id) end) as SI_delete_outlet,
count(case when close_outlet = 1 and delete_outlet is null then (outlets.id) end) as SI_closed_outlet,
min(case when parent_id is not null then date(outlets.created) end) as SI_Live_date,
DATE_FORMAT(outlets.created,  '%Y-%m-01') as Timeline
from outlets
left join orders on orders.id = outlets.order_id
where (outlets.created) between '2023-01-01 00:00:00' and '2024-12-31 23:59:59' 
group by parent_id, Timeline ;
'''

In [ ]:
gmb_query = '''
Select 
master_outlet_id,
count(case when is_published = 1 then outlet_id end) as GMB_Live,
count(case when is_published = 0 then outlet_id end) as Pending_GMB_Live,
min(date(created)) as GMB_Live_date,
DATE_FORMAT(created,  '%Y-%m-01') as Timeline
from google_location_datas
where created between '2023-01-01 00:00:00' and '2024-12-31 23:59:59' 
group by master_outlet_id, Timeline
having master_outlet_id is not null and master_outlet_id != '' and master_outlet_id !=0
'''

In [ ]:
brand_query =  '''
select 
wb.brand_name as wo_brand_name,
orders.order_code as order_code,
orders.id as order_id,
orders.wo_customer_id,
orders.order_status,
wa.total_locations AS Signed_location,
date(orders.created) as order_creation_date,
case when outlets.parent_id is null then outlets.id end  as master_outlet_id
from wo_customers wc
left join wo_brands wb on wb.wo_customer_id = wc.id
left join wo_agreements wa on wa.wo_customer_id = wc.id
left join orders on orders.wo_customer_id = wc.id
left join outlets on outlets.order_id = orders.id 
group by wo_customer_id;
'''

In [ ]:
raw = pd.read_sql(raw_query, con = spotlight)

In [ ]:
verified = pd.read_sql(verified_query, con = spotlight)

In [ ]:
si = pd.read_sql(SI_Query, con=sinterface)

In [ ]:
gmb = pd.read_sql(gmb_query, con= sinterface)

In [ ]:
brand = pd.read_sql(brand_query, con= sinterface)

In [ ]:
brand['master_outlet_id'] =  brand['master_outlet_id'].astype('float')

In [ ]:
data1 = pd.merge(raw, verified, on = ['order_code', 'Timeline'], how = 'outer')

In [ ]:
data2 = pd.merge(data1,si, on = ['order_code', 'Timeline'], how = 'outer')

In [ ]:
data3 = pd.merge(data2, gmb, on= ['master_outlet_id', 'Timeline'], how = 'outer' )

In [ ]:
data4 = pd.merge(data3, brand, on=['master_outlet_id', 'order_code'], how = 'outer')

In [ ]:
data4['Timeline'] = pd.to_datetime(data4['Timeline'])

In [ ]:
data4 = data4.sort_values(by=['master_outlet_id', 'Timeline'], ascending = True)

In [ ]:
#data4['master_outlet_id'] =data4['master_outlet_id'].astype('int')
data4['SI_live'].fillna(0, inplace=True)
data4['GMB_Live'].fillna(0, inplace=True)


data4['SI_live'] = data4['SI_live'] .astype('int')
data4['GMB_Live'] = data4['GMB_Live'] .astype('int')

In [ ]:
data4['SI_Live_diff'] = data4.groupby(['master_outlet_id'])['SI_live'].diff()
data4['GMB_Live_diff'] = data4.groupby(['master_outlet_id'])['GMB_Live'].diff()

In [ ]:
data4.columns

In [ ]:
data4.to_csv('verification_Live_loc_chanages.csv', index = False)

# Faltu ka

In [14]:
verification_data = '''
SELECT 
`name` as veri_brand_name,
upper(orderCode) as order_code,
count(r.id) as raw_location,
count(case when rec.call_status in (15,18,13,21,14,19,11,4,6,16,17) or rec.call_status in ('Closed','Duplicate', 'Not Cooperating', 'Not verifiable', 'Incomplete data', 'Toll-Free Number', 'Call back later','Not connected','Wrong Number', 'Language Issue', 'Location Pending')  then rec.id end) as unverifiable_location,
count(CASE WHEN (vr.call_status = 'Verification done' or vr.call_status = 5) THEN ( vr.raw_location_id) END) as verified_location,
count(CASE WHEN (vr.call_status = 'Closed' or vr.call_status = 15) THEN (vr.raw_location_id) END) as verified_closed_location,
DATE_FORMAT((case when r.is_uploded = 0 then (date(r.created_at)) else (date(JSON_EXTRACT(row_data, '$[0]')))  end),  '%Y-%m-01')as Timeline
FROM verification_myspotlight.raw_locations r
left join rectified_locations rec on rec.raw_location_id = r.id
left join verified_locations vr on vr.raw_location_id = r.id
inner join brands on brands.id = r.brand_id
where is_deleted = 0 and orderCode is not null and orderCode != ''
group by order_code, Timeline
having Timeline between '2023-01-01' and '2024-12-01';
'''

In [ ]:
si_append_query =  '''
select 
wb.brand_name as wo_brand_name,
orders.order_code as order_code,
orders.id as order_id,
orders.wo_customer_id,
COUNT(CASE WHEN delete_outlet IS NULL AND close_outlet IS NULL and parent_id is not null THEN outlets.id END) AS SI_live,
count(case when delete_outlet = 1 and close_outlet is  null then (outlets.id) end) as SI_delete_outlet,
count(case when close_outlet = 1 and delete_outlet is null then (outlets.id) end) as SI_closed_outlet,
count(case when is_published = 1 then outlet_id end) as GMB_Live,
min(case when parent_id is not null then outlets.created end) as SI_Live,
min(date(gd.created)) as GMB_Live,
wa.total_locations as Signed_location,
date(orders.created) as order_creation_date,
'2023-01-01'as Timeline
from wo_customers wc
left join wo_brands wb on wb.wo_customer_id = wc.id
left join wo_agreements wa on wa.wo_customer_id = wc.id
left join wo_pricing_details wp on wp.wo_customer_id = wc.id
left join orders on orders.wo_customer_id = wc.id
left join outlets on outlets.order_id = orders.id 
left join google_location_datas gd on gd.master_outlet_id = outlets.parent_id and gd.outlet_id = outlets.id
where orders.created <= '2023-01-01 00:00:00' 
group by wo_customer_id, Timeline;
'''
jan_data = pd.read_sql(si_append_query, con=sinterface)

In [15]:
verific_data = pd.read_sql(verification_data, con=spotlight)

In [16]:
verific_data['Timeline'] = pd.to_datetime(verific_data['Timeline'])

In [ ]:
verific_data


In [5]:
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)


In [6]:
final_df = pd.DataFrame()
dataframes = []


In [7]:
current_date = start_date


In [9]:
dataframes = []
while current_date <= end_date:
    # Get the first day of the current month
    month_start = pd.Timestamp(current_date).replace(day=1)
    
    # Get the last day of the current month using MonthEnd
    month_end = (month_start + pd.DateOffset(months=1)) - pd.DateOffset(days=1)

    # Create dynamic SQL query
    query = f"""
    SELECT 
        wb.brand_name AS wo_brand_name,
        wm.order_code AS order_code,
        orders.id AS order_id,
        outlets.parent_id as masater_outlet_id,
        wc.wo_customer_id as wc_wo_customer_id,
        orders.wo_customer_id as orders_wo_customer_id,
        orders.order_status,
        COUNT(CASE WHEN delete_outlet IS NULL AND close_outlet IS NULL and parent_id is not null THEN outlets.id END) AS SI_live,
        COUNT(CASE WHEN delete_outlet = 1 AND close_outlet IS NULL THEN outlets.id END) AS SI_delete_outlet,
        COUNT(CASE WHEN close_outlet = 1 AND delete_outlet IS NULL THEN outlets.id END) AS SI_closed_outlet,
        COUNT(CASE WHEN is_published = 1 THEN outlet_id END) AS GMB_Live,
        (case when wp.fixed_price_amount is null then wa.monthly_revenue else wp.fixed_price_amount end) as Contracted_Revenue,
        wp.currency_for_pricing as Currenncy,
        min(case when parent_id is not null then outlets.created end) as SI_Live_date,
        MIN(DATE(gd.created)) AS GMB_Live_date,
        wa.total_locations AS Signed_location,
        DATE(orders.created) AS order_creation_date
        -- DATE_FORMAT(orders.created, '%Y-%m-01') AS Timeline
    FROM wo_customers wc
    LEFT JOIN wo_brands wb ON wb.wo_customer_id = wc.id
    LEFT JOIN wo_agreements wa ON wa.wo_customer_id = wc.id
    LEFT JOIN wo_pricing_details wp ON wp.wo_customer_id = wc.id
    left join wo_miscellaneouses wm on wm.wo_customer_id = wc.id
    LEFT JOIN orders ON orders.wo_customer_id = wb.wo_customer_id
    LEFT JOIN outlets ON outlets.order_id = orders.id and outlets.created <= '{month_end.strftime('%Y-%m-%d 00:00:00')}'
    LEFT JOIN google_location_datas gd ON gd.master_outlet_id = outlets.parent_id AND gd.outlet_id = outlets.id and gd.created <= '{month_end.strftime('%Y-%m-%d 00:00:00')}'
    WHERE orders.created <= '{month_end.strftime('%Y-%m-%d 00:00:00')}'   
    GROUP BY orders.wo_customer_id;
    """

    # Fetch the result and append to the final DataFrame
    month_df = pd.read_sql(query, con=sinterface)
    print(month_df.shape)
    if not month_df.empty:
        # Add Timeline column with the format 'Y-m-01'
        month_df = month_df.assign(Timeline=month_start.strftime('%Y-%m-01'))
        dataframes.append(month_df)
        print(f"Data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')} appended successfully")
    else:
        print(f"No data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')}")

    dataframes.append(month_df)
    # Move to the next month
    current_date = month_end + pd.DateOffset(days=1)  # Start of next month


(140, 17)
Data for 2023-01-01 to 2023-01-31 appended successfully
(147, 17)
Data for 2023-02-01 to 2023-02-28 appended successfully
(160, 17)
Data for 2023-03-01 to 2023-03-31 appended successfully
(167, 17)
Data for 2023-04-01 to 2023-04-30 appended successfully
(185, 17)
Data for 2023-05-01 to 2023-05-31 appended successfully
(197, 17)
Data for 2023-06-01 to 2023-06-30 appended successfully
(210, 17)
Data for 2023-07-01 to 2023-07-31 appended successfully
(221, 17)
Data for 2023-08-01 to 2023-08-31 appended successfully
(229, 17)
Data for 2023-09-01 to 2023-09-30 appended successfully
(241, 17)
Data for 2023-10-01 to 2023-10-31 appended successfully
(251, 17)
Data for 2023-11-01 to 2023-11-30 appended successfully
(264, 17)
Data for 2023-12-01 to 2023-12-31 appended successfully
(274, 17)
Data for 2024-01-01 to 2024-01-31 appended successfully
(283, 17)
Data for 2024-02-01 to 2024-02-29 appended successfully
(297, 17)
Data for 2024-03-01 to 2024-03-31 appended successfully
(309, 17)


In [10]:
final_df = pd.concat(dataframes, ignore_index=True)

In [11]:
final_df['Timeline'] =  pd.to_datetime(final_df['Timeline'], format = '%Y-%m-%d')

In [12]:
final_df = final_df.drop_duplicates()

In [17]:
final_verif = pd.merge(verific_data, final_df, how = 'outer', left_on = ['order_code', 'Timeline'], right_on = ['order_code', 'Timeline'])

In [18]:
final_verif.columns

Index(['veri_brand_name', 'order_code', 'raw_location',
       'unverifiable_location', 'verified_location',
       'verified_closed_location', 'Timeline', 'wo_brand_name', 'order_id',
       'masater_outlet_id', 'wc_wo_customer_id', 'orders_wo_customer_id',
       'order_status', 'SI_live', 'SI_delete_outlet', 'SI_closed_outlet',
       'GMB_Live', 'Contracted_Revenue', 'Currenncy', 'SI_Live_date',
       'GMB_Live_date', 'Signed_location', 'order_creation_date'],
      dtype='object')

In [19]:
final_verif = final_verif.sort_values(by=['order_id', 'Timeline'], ascending = True)

In [20]:
final_verif['SI_Live_diff'] = final_verif.groupby(['order_id'])['SI_live'].diff()
final_verif['GMB_Live_diff'] = final_verif.groupby(['order_id'])['GMB_Live'].diff()

In [21]:
final_verif.to_csv('final_verif_new_location.csv')

In [ ]:
final_df['SI_live'].unique()

In [ ]:
final_df.to_csv('signed_live_funnel_log.csv', index = False)

In [ ]:
len(final_df['order_id'].unique())

In [ ]:
#final_df

In [ ]:
raw_data =  pd.read_sql(raw_query, con=spotlight)

In [ ]:
verified_data = pd.read_sql(verified_query, con=spotlight)

In [ ]:
SI_data = pd.read_sql(SI_Query, con=sinterface)

In [ ]:
gmb_data = pd.read_sql(gmb_query, con=sinterface)

# Starts from here

In [ ]:
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)


In [ ]:
current_date = start_date
final_df_si_gmb = pd.DataFrame()


In [ ]:
final_df_si_gmb = pd.DataFrame()
final_df_gmb = pd.DataFrame()
final_df_si = pd.DataFrame()
final_df_callonly = pd.DataFrame()
final_df_hyper_x = pd.DataFrame()

# SI_GMB

In [ ]:
final_df_si_gmb = pd.DataFrame()

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
current_date = start_date

dataframes_sigmb = []
while current_date <= end_date:
    # Get the first day of the current month
    month_start = pd.Timestamp(current_date).replace(day=1)
    
    # Get the last day of the current month using MonthEnd
    month_end = (month_start + pd.DateOffset(months=1)) - pd.DateOffset(days=1)

    query_si_gmb = f'''
    SELECT 
        wb.brand_name AS wo_brand_name,
        wm.order_code AS order_code,
        orders.id AS order_id,
        outlets.parent_id as master_outlet_id,
        orders.order_status,
        COUNT(CASE WHEN delete_outlet IS NULL AND close_outlet IS NULL and parent_id is not null THEN outlets.id END) AS SI_live,
        COUNT(CASE WHEN is_published = 1 THEN outlet_id END) AS GMB_Live
    FROM wo_customers wc
    LEFT JOIN wo_brands wb ON wb.wo_customer_id = wc.id
    left join wo_miscellaneouses wm on wm.wo_customer_id = wc.id
    LEFT JOIN orders ON orders.wo_customer_id = wb.wo_customer_id
    LEFT JOIN outlets ON outlets.order_id = orders.id AND  outlets.created <='{month_end.strftime('%Y-%m-%d 23:59:59')}'
    LEFT JOIN google_location_datas gd ON gd.master_outlet_id = outlets.parent_id AND gd.outlet_id = outlets.id and gd.created <= '{month_end.strftime('%Y-%m-%d 23:59:59')}'
    WHERE orders.created <= '{month_end.strftime('%Y-%m-%d 23:59:59')}' and orders.wo_customer_id not in (1,331,332,339,257,262,295,353)
    GROUP BY orders.wo_customer_id;
    '''
    
    month_df_sigmb = pd.read_sql(query_si_gmb, con=sinterface)
    print(month_df_sigmb.shape)
    if not month_df_sigmb.empty:
        # Add Timeline column with the format 'Y-m-01'
        month_df_sigmb = month_df_sigmb.assign(Timeline=month_start.strftime('%Y-%m-01'))
        dataframes_sigmb.append(month_df_sigmb)
        print(f"Data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')} appended successfully")
    else:
        print(f"No data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')}")

    dataframes_sigmb.append(month_df_sigmb)
    # Move to the next month
    current_date = month_end + pd.DateOffset(days=1)  # Start of next month


In [ ]:
final_df_si_gmb = pd.concat(dataframes_sigmb, ignore_index=True)

In [ ]:
final_df_si_gmb = final_df_si_gmb.drop_duplicates(subset = ['master_outlet_id', 'Timeline'])

In [ ]:
final_df_si_gmb

# GMB

In [ ]:
final_df_gmb = pd.DataFrame()

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
current_date = start_date
dataframes_gmb = []
while current_date <= end_date:
    # Get the first day of the current month
    month_start = pd.Timestamp(current_date).replace(day=1)
    
    # Get the last day of the current month using MonthEnd
    month_end = (month_start + pd.DateOffset(months=1)) - pd.DateOffset(days=1)

    query_gmb_only = f'''
    SELECT    
    	gd.master_outlet_id,
        SUM(gd.total_phone_call) AS GMB_calls
        from  sinterface.gmb_insight_outlet_datas gd
    where gd.cron_date between '{month_start.strftime('%Y-%m-%d')}' and '{month_end.strftime('%Y-%m-%d')}'
    GROUP BY master_outlet_id;
    '''
    month_df_gmb = pd.read_sql(query_gmb_only, con=sinterface)
    print(month_df_gmb.shape)
    if not month_df_gmb.empty:
        # Add Timeline column with the format 'Y-m-01'
        month_df_gmb = month_df_gmb.assign(Timeline=month_start.strftime('%Y-%m-01'))
        dataframes_gmb.append(month_df_gmb)
        print(f"Data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')} appended successfully")
    else:
        print(f"No data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')}")

    dataframes_gmb.append(month_df_gmb)
    # Move to the next month
    current_date = month_end + pd.DateOffset(days=1)  # Start of next month

In [ ]:
final_df_gmb = pd.concat(dataframes_gmb, ignore_index=True)

In [ ]:
final_df_gmb

# SI

In [ ]:
final_df_si = pd.DataFrame()

start_date = datetime(2024, 10, 1)
end_date = datetime(2024, 12, 31)
current_date = start_date

dataframes_si = []
while current_date <= end_date:
    # Get the first day of the current month
    month_start = pd.Timestamp(current_date).replace(day=1)
    
    # Get the last day of the current month using MonthEnd
    month_end = (month_start + pd.DateOffset(months=1)) - pd.DateOffset(days=1)

    query_si_only = f'''
    SELECT    
    	gs.parent_id as master_outlet_id,
    	SUM(CASE WHEN gs.eventAction IN ('click_to_call' , 'ev_click_to_call') THEN gs.totalEvents ELSE 0 END) AS 'SI_calls'
        from ga_tracking_clickevent_datas gs  
        where    gs.event_date between  '{month_start.strftime('%Y-%m-%d')}' and '{month_end.strftime('%Y-%m-%d')}'
    GROUP BY master_outlet_id;
    '''
    month_df_si = pd.read_sql(query_si_only, con=sinterface)
    print(month_df_si.shape)
    if not month_df_si.empty:
        # Add Timeline column with the format 'Y-m-01'
        month_df_si = month_df_si.assign(Timeline=month_start.strftime('%Y-%m-01'))
        dataframes_si.append(month_df_si)
        print(f"Data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')} appended successfully")
    else:
        print(f"No data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')}")

    dataframes_si.append(month_df_si)
    # Move to the next month
    current_date = month_end + pd.DateOffset(days=1)  # Start of next month

In [ ]:
final_df_si = pd.concat(dataframes_si, ignore_index=True)

In [ ]:
final_df_si

# VMN

In [ ]:
query_vmn = '''
Select parent_id as master_outlet_id, 
count(id) as vmn_activated 
from
outlets 
where id in (
select distinct outlet_id
FROM customer_call_record_logs)
and delete_outlet is null and close_outlet is null
group by master_outlet_id
'''
final_data_vmn = pd.read_sql(query_vmn, con=sinterface)

In [ ]:
final_data_vmn[final_data_vmn['master_outlet_id'] == 321091]

# Calls Only

In [ ]:
final_df_callonly = pd.DataFrame()

start_date = datetime(2024, 10, 1)
end_date = datetime(2024, 12, 31)
current_date = start_date

dataframes_call_only = []
while current_date <= end_date:
    # Get the first day of the current month
    month_start = pd.Timestamp(current_date).replace(day=1)
    
    # Get the last day of the current month using MonthEnd
    month_end = (month_start + pd.DateOffset(months=1)) - pd.DateOffset(days=1)

    query_call_only = f'''
    Select  cl.master_outlet_id,
        count(cl.id) as VMN_Calls,
        AVG(TIMESTAMPDIFF(MINUTE, call_start_time, call_end_time)) AS Avg_duration
        from  customer_call_record_logs cl 
    where call_date_time between '{month_start.strftime('%Y-%m-%d 00:00:00')}' and  '{month_end.strftime('%Y-%m-%d 23:59:59')}' and call_status in ('ANSWERED','Connected','Voicemail') 
    group by cl.master_outlet_id;
    '''
    month_df_call = pd.read_sql(query_call_only, con=sinterface)
    print(month_df_call.shape)
    if not month_df_call.empty:
        # Add Timeline column with the format 'Y-m-01'
        month_df_call = month_df_call.assign(Timeline=month_start.strftime('%Y-%m-01'))
        dataframes_call_only.append(month_df_call)
        print(f"Data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')} appended successfully")
    else:
        print(f"No data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')}")

    dataframes_call_only.append(month_df_call)
    # Move to the next month
    current_date = month_end + pd.DateOffset(days=1)  # Start of next month

In [ ]:
final_df_callonly = pd.concat(dataframes_call_only, ignore_index=True)

In [ ]:
final_df_callonly

# Hyper X

In [ ]:
final_df_hyper_x = pd.DataFrame()

start_date = datetime(2024, 10, 1)
end_date = datetime(2024, 12, 31)
current_date = start_date

dataframes_call_X = []
while current_date <= end_date:
    # Get the first day of the current month
    month_start = pd.Timestamp(current_date).replace(day=1)
    
    # Get the last day of the current month using MonthEnd
    month_end = (month_start + pd.DateOffset(months=1)) - pd.DateOffset(days=1)

    query_hyper_x = f'''
     Select  cl.master_outlet_id,
         count(cl.id) as VMN_Hyperx_Calls,
     COUNT(DISTINCT cl.outlet_id) AS Hyperx_active,
        AVG(TIMESTAMPDIFF(MINUTE, call_start_time, call_end_time)) AS Avg_duration_Hyperx
        from  customer_call_record_logs cl 
        inner join call_recording_details cd on cd.customer_call_record_id = cl.id and cd.master_outlet_id = cl.master_outlet_id and cl.outlet_id = cd.outlet_id
    where call_date_time between  '{month_start.strftime('%Y-%m-%d 00:00:00')}' and  '{month_end.strftime('%Y-%m-%d 23:59:59')}' and call_status = 'Connected' and analytic_type = 'call' 
    group by cl.master_outlet_id;

    '''
    month_df_call_X = pd.read_sql(query_hyper_x, con=sinterface)
    print(month_df_call_X.shape)
    if not month_df_call_X.empty:
        # Add Timeline column with the format 'Y-m-01'
        month_df_call_X = month_df_call_X.assign(Timeline=month_start.strftime('%Y-%m-01'))
        dataframes_call_X.append(month_df_call_X)
        print(f"Data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')} appended successfully")
    else:
        print(f"No data for {month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')}")

    dataframes_call_X.append(month_df_call_X)
    # Move to the next month
    current_date = month_end + pd.DateOffset(days=1)  # Start of next month

In [ ]:
final_df_hyper_x = pd.concat(dataframes_call_X, ignore_index=True)

In [ ]:
data1 = pd.merge(final_df_si_gmb, final_df_gmb, how = 'left', on= ['master_outlet_id', 'Timeline'] )

In [ ]:
data1 = data1.drop_duplicates()

In [ ]:
data2 = pd.merge(data1, final_df_si, how = 'left', on= ['master_outlet_id', 'Timeline'])
data2 = data2.drop_duplicates()

In [ ]:
data3 = pd.merge(data2, final_data_vmn, how = 'left', on= ['master_outlet_id'])
data3 = data3.drop_duplicates()

In [ ]:
data4 = pd.merge(data3, final_df_callonly,  how = 'left', on= ['master_outlet_id', 'Timeline'])
data4 = data4.drop_duplicates()

In [ ]:
data5 = pd.merge(data4, final_df_hyper_x ,   how = 'left', on= ['master_outlet_id', 'Timeline'])
data5 = data5.drop_duplicates()

In [ ]:
data5['Timeline'].unique()

In [ ]:
data5.fillna(0, inplace = True)

data5['order_id'] = data5['order_id'].astype('int')
data5['master_outlet_id'] = data5['master_outlet_id'].astype('int')
data5['SI_live'] = data5['SI_live'].astype('int')
data5['GMB_Live'] = data5['GMB_Live'].astype('int')

data5['Timeline'] = pd.to_datetime(data5['Timeline'])

data5['GMB_calls'] = data5['GMB_calls'].astype('int')
data5['SI_calls'] = data5['SI_calls'].astype('int')
data5['vmn_activated'] = data5['vmn_activated'].astype('int')

data5['Avg_duration'] = data5['Avg_duration'].round(2)
data5['Hyperx_active'] = data5['Hyperx_active'].astype('int')
data5['Avg_duration_Hyperx'] = data5['Avg_duration_Hyperx'].round(2)


In [ ]:
data5['VMN_Acti%'] = data5.apply(
    lambda row: row['vmn_activated'] / max(row['GMB_Live'], row['SI_live']) if max(row['GMB_Live'], row['SI_live']) > 0 else 0,
    axis=1
)




In [ ]:
data5['VMN_Acti%'] = data5['VMN_Acti%'].round(2)

In [ ]:
data5.to_csv('VMN_related_data.csv')